<a href="https://colab.research.google.com/github/Apekshaa2908/Enhancing-E-Commerce-Agility-With-Advanced-ETL-Pipeline/blob/main/Step_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "Comment": "E-commerce ETL Pipeline Orchestration",
  "StartAt": "Lambda Invoke",
  "States": {
    "Lambda Invoke": {
      "Type": "Task",
      "Resource": "arn:aws:states:::lambda:invoke",
      "OutputPath": "$.Payload",
      "Parameters": {
        "Payload.$": "$",
        "FunctionName": "arn:aws:lambda:ap-southeast-2:590183855618:function:FinalProject:$LATEST"
      },
      "Retry": [
        {
          "ErrorEquals": [
            "Lambda.ServiceException",
            "Lambda.AWSLambdaException",
            "Lambda.SdkClientException",
            "Lambda.TooManyRequestsException"
          ],
          "IntervalSeconds": 1,
          "MaxAttempts": 3,
          "BackoffRate": 2
        }
      ],
      "Next": "Glue Job Has Started"
    },
    "Glue Job Has Started": {
      "Type": "Wait",
      "Seconds": 300,
      "Next": "Success or Failure"
    },
    "Success or Failure": {
      "Type": "Choice",
      "Choices": [
        {
          "Variable": "$.JobRun.JobRunState",
          "StringEquals": "SUCCEEDED",
          "Next": "The Data is Pushed to redshift"
        },
        {
          "Variable": "$.JobRun.JobRunState",
          "StringEquals": "FAILED",
          "Next": "The Glue job has failed"
        }
      ]
    },
    "The Data is Pushed to redshift": {
      "Type": "Task",
      "Resource": "arn:aws:states:::sns:publish",
      "Parameters": {
        "TopicArn": "arn:aws:sns:ap-southeast-2:590183855618:FinalProject",
        "Message": {
          "Message": "The Glue job has succeded."
        }
      },
      "End": true
    },
    "The Glue job has failed": {
      "Type": "Task",
      "Resource": "arn:aws:states:::sns:publish",
      "Parameters": {
        "TopicArn": "arn:aws:sns:ap-southeast-2:590183855618:FinalProject",
        "Message": {
          "Message": "The Glue job failed."
        }
      },
      "End": true
    }
  }
}